#Importing required Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#Aircraft Fuselage Parameters from the user


In [4]:

class Environment:
    def __init__(self):
        self.environment_set:bool = False
        self.temperature_sea_level: float = None
        self.Reynolds_number: float = None
        self.altitude: float = None
        self.pressure_sea_level: float = None
        self.Lapse_rate_troposphere: float = None
        self.Specific_gas_constant: float = 287.053   # constant
        self.gravitational_acceleration: float = 9.80665  # constant
        self.specific_heat_constant: float = 1.4      # constant
        self.wind_velocity: float = None
        self.ISA_OFFSET: float = None


    def get_atmosphere_parameters(self):
        self.environment_set = True
        self.temperature_sea_level = float(input("Please enter the sea level temperature (K): "))
        self.Reynolds_number = float(input("Please enter the Reynolds number: "))
        self.altitude = float(input("Please enter the altitude (m): "))
        self.pressure_sea_level = float(input("Please enter the pressure at sea level (Pa): "))
        self.Lapse_rate_troposphere = float(input("Please enter the lapse rate of the troposphere (K/m): "))
        self.wind_velocity = float(input("Please enter the wind velocity (m/s): "))
        self.ISA_OFFSET = float(input("Please enter the ISA offset (K): "))
        
    def Temp(self, altitude):
        if not self.environment_set:
            raise ValueError("Environment parameters have not been set.")
        T_new = self.temperature_sea_level + self.Lapse_rate_troposphere * altitude + self.ISA_OFFSET
        return T_new
    def Pressure(self, T_new):
        if not self.environment_set:
            raise ValueError("Environment parameters have not been set.")
        P_new = self.pressure_sea_level * ((T_new / self.temperature_sea_level) ** (self.gravitational_acceleration / self.Lapse_rate_troposphere * self.Specific_gas_constant))
        return P_new

    def speed_of_sound(self, temperature):
        if not self.environment_set:
            raise ValueError("Environment parameters have not been set.")
        return np.sqrt(1.4 * 287 * temperature)

    def mach_number(self, velocity, speed_of_sound):
        if not self.environment_set:
            raise ValueError("Environment parameters have not been set.")
        return velocity / speed_of_sound

    def density(self, temperature, pressure):
        if not self.environment_set:
            raise ValueError("Environment parameters have not been set.")
        return pressure / (287 * temperature)

In [ ]:


class Rotor:
    def __init__(self):
        self.parameters_set:bool = False
        

    def get_rotor_parameters(self):
        self.parameters_set = True
        self.number_of_blades = int(input("Please enter the number of rotors: "))
        self.omega = float(input("Please enter the angular velocity of the rotor (rad/s): "))#rotor_omega=2*np.pi*rotor_rpm/60
        self.blade_mass = float(input("Please enter the mass of the rotors (kg): "))
        self.NACA_for_airfoil = input("Please enter the NACA number of the airfoil: ")
        self.radius_of_rotors = float(input("Please enter the radius of the rotors (m): "))
        self.root_cutout = float(input("Please enter the root cutout of the rotors (m): "))
        self.angle_of_attack = (np.pi/180) * float(input("Please enter the angle of attack (degrees): "))
        self.root_chord = float(input("Please enter the root chord (m): "))
        self.tip_chord = float(input("Please enter the tip chord (m): "))
        self.root_pitch = (np.pi/180) * float(input("Please enter the root pitch (degrees): "))
        self.slope_pitch = float(input("Please enter the slope of the pitch: "))
        # self.climb_velocity = float(input("Please enter the desired climb velocity (m/s): "))
    def get_chord_length(self, x):
        chord_length_x = (self.root_chord - ((self.root_chord - self.tip_chord) / (self.radius_of_rotors - self.root_cutout)) * x)
        return chord_length_x

    def pitch(self, x):  # Basically involving twists
        local_pitch = self.root_pitch + self.slope_pitch * x
        return local_pitch

    def elemental_inflow_ratio(self, r_distance, climb_velocity, max_iter=100):
        lambda_c=climb_velocity/(self.omega*self.radius_of_rotors)
        sigma=(self.number_of_blades*(0.5*(self.root_chord+self.tip_chord)*(self.radius_of_rotors- self.root_cutout)))/(np.pi*(self.radius_of_rotors**2))
        cl=0.7
        a=cl/self.angle_of_attack

        # Initial guess values to start with
        F = 1.1

        lambda_inflow = 0.2

        for i in range(max_iter):
            theta = self.pitch(r_distance)

            #  geting the  Cl (lift coefficient)


            # Update lambda_inflow using current F
            new_lambda_inflow = (
                np.sqrt(((sigma * a) / (16 * F) - (lambda_c / 2))**2
                        + (sigma * a) / (8 * F) * theta * (r_distance / self.radius_of_rotors))
                - ((sigma * a) / (16 * F) - (lambda_c / 2))
            )


            # Update F using the new lambda_inflow (Gauss–Seidel step)
            f = (self.number_of_blades / 2) * ((1 - r_distance / self.radius_of_rotors) / new_lambda_inflow)

            new_F = (2 / np.pi) * np.arccos(np.exp(-f))
            
            F = new_F
            lambda_inflow = new_lambda_inflow

            # Check convergence
            """if abs(new_lambda_inflow - lambda_inflow) < tol and abs(new_F - F) < tol:
                return new_lambda_inflow, new_F

            # Update values for next iteration
            lambda_inflow, F = new_lambda_inflow, new_F"""

        return lambda_inflow, F


    def elemental_thrust(self, r_distance, dr, climb_velocity):
        c, b = self.elemental_inflow_ratio(r_distance, climb_velocity)
        density = 5
        dT = 4 * np.pi * density * r_distance * (c * self.omega * self.radius_of_rotors) * ((c * self.omega * self.radius_of_rotors) - climb_velocity) * dr
        return dT

    def total_thrust(self, climb_velocity,  n_divisions=50):
        if self.parameters_set == False:
            raise ValueError("Rotor parameters have not been set.")
        
        r=np.linspace(self.root_cutout, self.radius_of_rotors, n_divisions)
        delta_r=r[1]-r[0]
        store_dT=[]
        for i in range(1, n_divisions-1):
            store_dT.append(self.elemental_thrust(r[i], delta_r, climb_velocity))
        net_thrust = self.number_of_blades * np.sum(store_dT)
        return net_thrust



In [6]:

class Helicopter: 
    def __init__(self, environment: Environment):
        self.environment: Environment = environment
        self.fuselage_set: bool = False
        self.engine_set:bool = False
        self.rotor_set: bool = False
        self.climb_velocity:float = 0
        self.rotor:Rotor = None


    def get_fuselage_parameters(self):
        self.fuselage_set = True
        self.fuselage_mass = float(input("Please enter the fuselage mass (kg): "))
        self.fuselage_length = float(input("Please enter the fuselage length (m): "))
        self.fuselage_width = float(input("Please enter the fuselage width (m): "))
        self.fuselage_height = float(input("Please enter the fuselage height (m): "))
        self.fuselage_cl = float(input("Please enter the fuselage lift coefficient (Cl): "))
        self.fuselage_cd = float(input("Please enter the fuselage drag coefficient (Cd): "))
        
        
    def get_rotor_parameters(self):
        self.rotor = Rotor()
        self.rotor_set = True
        self.rotor.get_rotor_parameters()
        self.climb_velocity = float(input("Please enter the desired climb velocity (m/s): "))
        
    def get_engine_parameters(self):
        self.engine_set = True
        self.engine_mass = float(input("Please enter the mass of the engine (kg): "))
        self.engine_BSFC = float(input("Please enter the BSFC of the engine (kg/kWh): "))
        self.engine_shaft_power_conversion_efficiency = float(input("Please enter the shaft power conversion efficiency: "))
        self.engine_fractional_power_tail_rotor = float(input("Please enter the fractional power of the tail rotor: "))

In [ ]:


# #fuselage parameters
# def get_fuselage_parameters():
#   fuselage={}
#   fuselage["mass"]=float(input("Please enter the mass of the fuselage:"))
#   fuselage["length"]=float(input("Please enter the length of the fuselage:"))
#   fuselage["width"]=float(input("Please enter the width of the fuselage:"))
#   fuselage["height"]=float(input("Please enter the height of the fuselage:"))
#   fuselage["CL"]=float(input("Please enter the CL of the fuselage:"))
#   fuselage["CD"]=float(input("Please enter the CD of the fuselage:"))
#   return fuselage

# Aircraft engine parameters

In [ ]:
# #Engine parameters
# def get_engine_parameters():
#   engine={}
#   engine["mass"]=float(input("Please enter the mass of the engine:"))
#   engine["BSFC"]=float(input("Please enter the BSFC of the engine:"))
#   engine["shaft power conversion efficiency"]=float(input("Please enter the shaft power conversion efficiency:"))
#   engine["fractional power tail rotor"]=float(input("Please enter the fractional power of the tail rotor:"))
#   return engine

#Rotor Assembly characteristics


In [ ]:
# def get_rotor_parameters():
#     rotor = {}
#     rotor["number_of_blades"] = int(input("Please enter the number of rotors: "))
#     rotor["rotor_omega"] = float(input("Please enter the angular velocity of the rotor (rad/s): "))#rotor_omega=2*np.pi*rotor_rpm/60
#     rotor["blade_mass"] = float(input("Please enter the mass of the rotors (kg): "))
#     rotor["NACA_for_airfoil"] = input("Please enter the NACA number of the airfoil: ")
#     rotor["radius_of_rotors"] = float(input("Please enter the radius of the rotors (m): "))
#     rotor["root_cutout"] = float(input("Please enter the root cutout of the rotors (m): "))
#     rotor["angle_of_attack"] = float(input("Please enter the angle of attack (degrees): "))
#     rotor["root_chord"] = float(input("Please enter the root chord (m): "))
#     rotor["tip_chord"] = float(input("Please enter the tip chord (m): "))
#     rotor["root_pitch"] = float(input("Please enter the root pitch (degrees): "))
#     rotor["slope_pitch"] = float(input("Please enter the slope of the pitch: "))
#     rotor["climb_velocity"] = float(input("Please enter the desired climb velocity (m/s): "))
#     return rotor

In [ ]:
# rotor_params = get_rotor_parameters()

# number_of_blades = rotor_params["number_of_blades"]
# rotor_omega = rotor_params["rotor_omega"]
# blade_mass = rotor_params["blade_mass"]
# NACA_for_airfoil = rotor_params["NACA_for_airfoil"]
# radius_of_rotors = rotor_params["radius_of_rotors"]
# root_cutout = rotor_params["root_cutout"]
# angle_of_attack = rotor_params["angle_of_attack"]
# root_chord = rotor_params["root_chord"]
# tip_chord = rotor_params["tip_chord"]
# root_pitch = rotor_params["root_pitch"]
# slope_pitch = rotor_params["slope_pitch"]
# climb_velocity = rotor_params["climb_velocity"]

Please enter the number of rotors: 5
Please enter the angular velocity of the rotor (rad/s): 200
Please enter the mass of the rotors (kg): 50
Please enter the NACA number of the airfoil: 2412
Please enter the radius of the rotors (m): 2
Please enter the root cutout of the rotors (m): 0.2
Please enter the angle of attack (degrees): 5
Please enter the root chord (m): 0.5
Please enter the tip chord (m): 0.6
Please enter the root pitch (degrees): 1
Please enter the slope of the pitch: 0.1
Please enter the desired climb velocity (m/s): 10


#Setting atmospheric parameters

In [ ]:
# def get_atmosphere_parameters():
#     atmosphere = {}

#     atmosphere["temperature_sea_level"] = float(input("Please enter the sea level temperature (K): "))
#     atmosphere["Reynolds_number"] = float(input("Please enter the Reynolds number: "))
#     atmosphere["altitude"] = float(input("Please enter the altitude (m): "))
#     atmosphere["pressure_sea_level"] = float(input("Please enter the pressure at sea level (Pa): "))
#     atmosphere["Lapse_rate_troposphere"] = float(input("Please enter the lapse rate of the troposphere (K/m): "))
#     atmosphere["Specific_gas_constant"] = 287.053   # constant
#     atmosphere["gravitational_acceleration"] = 9.80665  # constant
#     atmosphere["specific_heat_constant"] = 1.4      # constant
#     atmosphere["wind_velocity"] = float(input("Please enter the wind velocity (m/s): "))
#     atmosphere["ISA_OFFSET"] = float(input("Please enter the ISA offset (K): "))
#     return atmosphere



In [ ]:
# Example usage
# atmosphere_params = get_atmosphere_parameters()
# environment = Environment()
# environment.get_atmosphere_parameters()
# # Unpack into variables
# temperature_sea_level = atmosphere_params["temperature_sea_level"]
# Reynolds_number = atmosphere_params["Reynolds_number"]
# altitude = atmosphere_params["altitude"]
# pressure_sea_level = atmosphere_params["pressure_sea_level"]
# Lapse_rate_troposphere = atmosphere_params["Lapse_rate_troposphere"]
# Specific_gas_constant = atmosphere_params["Specific_gas_constant"]
# gravitational_acceleration = atmosphere_params["gravitational_acceleration"]
# specific_heat_constant = atmosphere_params["specific_heat_constant"]
# wind_velocity = atmosphere_params["wind_velocity"]
# ISA_OFFSET = atmosphere_params["ISA_OFFSET"]
# cl=0.7
# a=cl/angle_of_attack

Please enter the sea level temperature (K): 298
Please enter the Reynolds number: 20000
Please enter the altitude (m): 100
Please enter the pressure at sea level (Pa): 100000
Please enter the lapse rate of the troposphere (K/m): 5
Please enter the wind velocity (m/s): 1
Please enter the ISA offset (K): 2


# Computation of local parameters at a given altitude

In [ ]:
# def Temp(init_temp, ISA_Offset):
#   T_new= temperature_sea_level+  Lapse_rate_troposphere*altitude+ ISA_OFFSET
#   return T_new
# def Pressure(Pressure_sea_level, T_new, Temperature_sea_level, Lapse_rate_troposphere):
#   return Pressure_sea_level*((T_new/Temperature_sea_level)**(gravitational_acceleration/Lapse_rate_troposphere*Specific_gas_constant))
# def speed_of_sound(temperature):    # function to calculate the speed of sound
#   return np.sqrt(1.4*8.314*temperature)
# def mach_number(velocity, speed_of_sound):
#   return velocity/speed_of_sound
# def density(temperature, pressure):
#   return pressure/(287*temperature)

# Computing various coeffcients


In [ ]:

# def Cl_Cd_Cm(naca_number, alpha):
#     # Extract coefficients
#     cl=0.7
#     cd=0.01
#     cm=0.03
#     return cl,cd,cm
# # Function to get chord_lenth and pitch at a given location.
# def get_chord_length( chord_length ,radius_of_rotors,root_cutout, root_chord, tip_chord, root_pitch,x):
#   chord_length_x=(root_chord -((root_chord-tip_chord)/(radius_of_rotors-root_cutout))*x)
#   return chord_length_x
# def pitch(root_pitch,slope_pitch ,x): # Basically involving twists
#   local_pitch= root_pitch+ slope_pitch*x
#   return local_pitch

#Inflow for each blade section elementally

$$
\lambda =
\sqrt{
\left( \frac{\sigma a}{16F} - \frac{\lambda_c}{2} \right)^2
+ \frac{\sigma a}{8F} \, \theta \, \frac{r}{R}
}- \left( \frac{\sigma a}{16F} - \frac{\lambda_c}{2} \right)
$$

$$
\text{where,} \quad
\lambda_c = \frac{V}{\Omega R},
\quad
\lambda = \frac{V + v}{\Omega R}
$$

$$
F = \frac{2}{\pi}cos^{-1}(e^{-f})    ; where  f = \frac{b(1-\frac{r}{R})}{\lambda} \\
dT = 4\pi\rho rF(V+v)vdr = 4\pi\rho rF(\lambda ω r)(λωr - V)dr \\
T\ =\ b\int_{Rc}^{R}dT
$$

* we also include prandtl's tip loss model in the inflow ratio formula
* Since Lambda is dependent on F and f further is dependent on lamda we have to iteratively find out the value of Lambda until solutuion stabilizes.

#Define some useful parameters

In [ ]:
#  # since the blade is in the form of trapeziumlambda_c=climb_velocity/(rotor_omega*radius_of_rotors)
# sigma=(number_of_blades*(0.5*(root_chord+tip_chord)*(radius_of_rotors- root_cutout)))/(np.pi*(radius_of_rotors**2))

In [ ]:
# def elemental_inflow_ratio(r_distance, max_iter=100):
#     # Initial guess values to start with
#     F = 1.1

#     lambda_inflow = 0.2

#     for i in range(max_iter):
#         theta = pitch(root_pitch, slope_pitch, r_distance)

#         #  geting the  Cl (lift coefficient)


#         # Update lambda_inflow using current F
#         new_lambda_inflow = (
#             np.sqrt(((sigma * a) / (16 * F) - (lambda_c / 2))**2
#                     + (sigma * a) / (8 * F) * theta * (r_distance / radius_of_rotors))
#             - ((sigma * a) / (16 * F) - (lambda_c / 2))
#         )


#         # Update F using the new lambda_inflow (Gauss–Seidel step)
#         f = (number_of_blades / 2) * ((1 - r_distance / radius_of_rotors) / new_lambda_inflow)

#         new_F = (2 / np.pi) * np.arccos(np.exp(-f))


#         # Check convergence
#         """if abs(new_lambda_inflow - lambda_inflow) < tol and abs(new_F - F) < tol:
#             return new_lambda_inflow, new_F

#         # Update values for next iteration
#         lambda_inflow, F = new_lambda_inflow, new_F"""

#     return lambda_inflow, F


# Calculating the thrust from all rotors

In [ ]:
# # we now compute the elemental thrust
# def elemental_thrust(r_distance,dr):
#   c,b=elemental_inflow_ratio(r_distance)
#   density=5
#   dT=4*np.pi*density*r_distance*(c*rotor_omega*radius_of_rotors)*((c*rotor_omega*radius_of_rotors)-climb_velocity)*dr
#   return dT
# def total_thrust(n_divisions, number_of_blades):
#   r=np.linspace(root_cutout, radius_of_rotors, n_divisions)
#   delta_r=r[1]-r[0]
#   store_dT=[]
#   for i in range(1, n_divisions-1):
#     store_dT.append(elemental_thrust(r[i],delta_r))
#   net_thrust=number_of_blades*np.sum(store_dT)
#   return net_thrust

In [ ]:
# total_thrust(50,2)

np.float64(1364923.2722156523)

In [21]:
rotor = Rotor()
rotor.get_rotor_parameters()


In [ ]:
print(rotor.total_thrust(climb_velocity=10, n_divisions=50))

np.float64(3412308.1805391307)

3412308.1805391307


3282.806350011744